In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import h5py
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F

# from google.colab import files
# files.upload()

from google.colab import drive
drive.mount('/content/drive')

img_set = []
label_set = []
names_set = []

def load_train_dataset():
    global img_set, label_set, names_set
    train_dataset = h5py.File('/content/drive/My Drive/LLD-icon.hdf5', 'r')

    base_items = list(train_dataset.items())
    # print(base_items)
    lg = train_dataset.get('labels')
    # lg_items = list(lg.items())
    # print(lg_items)

    mdg = train_dataset.get('meta_data')
    # mdg_items = list(mdg.items())
    # print(mdg_items)

    img_set = np.array(train_dataset["data"][:50000])
    label_set = np.array(lg.get("ae_grayscale"))[:50000]
    names_set = np.array(mdg.get("names"))[:50000]

    img_set = torch.FloatTensor(img_set)
    label_set = label_set.astype(np.int)
    label_set = torch.LongTensor(label_set)

    print('---1', img_set.size())
    print(label_set[0:9])
    print(names_set[0:9])

    # for i in range(500,505):
    #   plt.imshow(np.transpose(img_set[i], (1, 2, 0)))
    #   plt.show()

    i = 0
    rows = 3
    cols = 3
    axes = []
    fig=plt.figure()
    for a in range(rows*cols):
        b = img_set[i]
        axes.append( fig.add_subplot(rows, cols, a+1) )

        subplot_title=(str(label_set[i])+' '+str(names_set[i], 'utf-8'))
        axes[-1].set_title(subplot_title)
         
        i += 1
        b = b.squeeze(0)
        b = b.numpy() # pyTorch tensor to numpy 
        b = np.moveaxis(b, 0, 2)
        plt.imshow(b.astype('uint8'))
    fig.tight_layout()    
    plt.show()

# load_train_dataset()

#================================= Test dataset ============================

def load_test_dataset():
    global img_set, label_set, names_set
    train_dataset = h5py.File('/content/drive/My Drive/LLD-icon.hdf5', 'r')

    base_items = list(train_dataset.items())
    # print(base_items)
    lg = train_dataset.get('labels')
    # lg_items = list(lg.items())
    # print(lg_items)

    mdg = train_dataset.get('meta_data')
    # mdg_items = list(mdg.items())
    # print(mdg_items)

    img_set = np.array(train_dataset["data"][50000:60000])
    label_set = np.array(lg.get("ae_grayscale"))[50000:60000]
    names_set = np.array(mdg.get("names"))[50000:60000]

    img_set = torch.FloatTensor(img_set)
    label_set = label_set.astype(np.int)
    label_set = torch.LongTensor(label_set)

    print(label_set)
    print(names_set[0:9])

load_test_dataset()
#==============================================================================

# Define a CNN

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        self.fc1 = nn.Linear(16 * 6 * 6, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, 200)
        self.fc4 = nn.Linear(200, 100)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

net = Net()
print(net)


### Define a Loss function and optimizer ###

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=1e-5, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr = 0.0001)


### Train the network ###

# loss_values=[]
# for epoch in range(10):  # loop over the dataset multiple times
#     running_loss = 0.0
#     for i in range(len(img_set)):
#         # get the inputs
#         inputs = img_set[i]
#         print('---2', inputs.size())
#         inputs = inputs.unsqueeze(0)

#         labels = label_set[i]
#         labels = torch.tensor([labels])

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         print('---2', inputs.size())
#         outputs = net(inputs)   # forward
        
#         #print(outputs)
#         # _, predicted = torch.max(outputs, 1)
#         # print(predicted)
        
#         loss = criterion(outputs, labels)
#         loss.backward()                             # backward
#         optimizer.step()                            # optimize

#         # print statistics
#         running_loss += loss.item()
#         if i % 50 == 49:    # print every 5 mini-batches
#             print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 5))
#             loss_values.append(running_loss)
#             running_loss = 0.0
# plt.plot(loss_values)
# plt.show()        
       
# print('Finished Training')

# save trained model:
PATH = '/content/drive/My Drive/lld_icons_recognition_model.pth'
#torch.save(net.state_dict(), PATH)

# load back the saved model
net = Net()
net.load_state_dict(torch.load(PATH))

## test
test_image = img_set[2]
test_image = test_image.unsqueeze(0)

outputs = net(test_image)
print(outputs)
_, predicted = torch.max(outputs, 1)

print('GroundTruth: ',label_set[2])
print('Predicted: ', predicted)


# network performs on the whole dataset
correct = 0
total = 0
with torch.no_grad():
    for i in range(len(img_set)):
        images = img_set[i]
        images = images.unsqueeze(0)

        labels = label_set[i]
        labels = torch.tensor([labels])

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the '+ str(len(img_set)) +' test images: %d %%' % (100 * correct / total))


# accuracy for all classes:
class_correct = list(0. for i in range(100))
class_total = list(0. for i in range(100))
with torch.no_grad():
  for i in range(len(img_set)):
        images = img_set[i]
        images = images.unsqueeze(0)

        labels = label_set[i]
        labels = torch. tensor([labels])

        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        
        class_correct[labels] += c.item()
        class_total[labels] += 1

for i in range(100):
    if class_total[i] != 0:
      print('Accuracy of %5s  %s  %3d/%3d  :    %2d %%' % (i, str(names_set[i]).rjust(35), class_correct[i], class_total[i], 100 * class_correct[i] / class_total[i]))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
tensor([42, 81, 18,  ..., 15, 60, 97])
[b'startupmanagement.org' b'mashadleatherstore.com' b'sekonic.com'
 b'ooopay.org' b'venjakob-moebel.de' b'gambettesbox.fr'
 b'polakuleczsiesam.pl' b'civiced.org' b'bestfm.ru']
Net(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=200, bias=True)
  (fc4): Linear(in_features=200, out_features=100, bias=True)
)
tensor([[-44.8561, -44.3563, -69.7696, -57.1968, -26.4748, -65.5800, -35.2170,
         -18.7126, -48.3753, -20.5466, -38.9215, -30.4174, -31.8363, -25.2967,
         -64.2231, -17.8837, -37.0965, 